# Automated ML

Importing all needed dependencies to complete the project.

In [4]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Dataset

### Overview
The dataset collects the records of females patients of age 21 and older from Pima Indian heritage. The dataset has a total of 768 entries. The objective is to predict if a patient has diabetes or not by evaluating certain diagnostic measurements. 
https://www.kaggle.com/mathchi/diabetes-data-set

Task
Predict the "Outcome" column based on the input features, either the patient has diabetes or not.

The dataset has nine features as follow:
- Pregnancies: Number pregnancy times (int).
- Glucose: Plasma glucose concentration level (int).
- BloodPressure: Diastolic blood pressure level in mm Hg(int).
- SkinThickness: skinfold thickness in mm(int).
- Insulin: two-hour serum insulin measured by mu U/ml(int).
- BMI: Body mass index (float).
- DiabetesPedigreeFunction: Diabetes pedigree function(float).
- Age: age in years 21 and above(int).
- Outcome: Target column 0 or 1, 0 = Not diabetes, 1 = diabetes(int).

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

In [6]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
#Display the first five records of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# Create CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_V2',
                                                           vm_priority = 'dedicated', 
                                                           max_nodes=30)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

Overview of the automl settings and configuration used for this experiment:

- "experiment_timeout_minutes": set to 30 minutes. The experiment will timeout after that period to avoid wasting resources.
- "max_concurrent_iterations": is set to 4. The max number of concurrent iterations to be run in parallel at the same time.
- "primary_metric" : is set to 'accuracy', which is a sutible metric for classification problems.
- "n_cross_validations": is set to 5, therefore the training and validation sets will be divided into five equal sets.
- "iterations": the number of iterations for the experiment is set to 20. It's a reasonable number and would provide the intendable result for the given dataset.
- compute_target: set to the project cluster to run the experiment.
- task: set to 'classification' since our target to predict whether the patient has diabetes or not.
- training_data: the loaded dataset for the project.
- label_column_name: set to the result/target colunm in the dataset 'Outcome' (0 or 1).
- enable_early_stopping: is enabled to terminate the experiment if the accuracy score is not showing improvement over time.
- featurization = is set to 'auto', it's an indicator of whether implementing a featurization step to preprocess/clean the dataset automatically or not. In our case, the preprocessing was applied for the numerical columns which normally involve treating missing values, cluster distance, the weight of evidence...etc.
- debug_log: errors will be logged into 'automl_errors.log'.


In [12]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 30,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 20
    
}

# Automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset,
                             label_column_name='Outcome',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )



In [13]:
# Submit experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_aef59d35-2dea-49cf-b96e-74638e4cd3c7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

The best model has resulted from the AutoML experiment from VotingEnsemble model. The Voting Ensemble model takes a majority vote of several algorithms which makes it surpass individual algorithms and minimize the bias. 

Use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_aef59d35-2dea-49cf-b96e-74638e4cd3c7,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

In [ ]:
remote_run

## Best Model

The cell below shows the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

In [ ]:
remote_run.get_metrics()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, we have registered the model, created an inference config and deployed the model as a web service.

In [ ]:
# Registring the best model
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')


In [ ]:
# from azureml.core.conda_dependencies import CondaDependencies

# # create environment
# environment = Environment(name="azure-env")
# conda_dep = CondaDependencies()

# # Nedded packages and scripts
# conda_dep.add_conda_package("pandas")
# conda_dep.add_conda_package("numpy")
# conda_dep.add_pip_package("scikit-learn")
# #conda_dep.add_conda_package("scikit-learn")
# conda_dep.add_pip_package("azureml-defaults")

# # Adding dependencies to the created environment
# environment.python.conda_dependencies=conda_dep

# Get automl environment with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [ ]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=False
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

In the cell below, we have sent a request to the web service to test it.

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {"data": [{"Pregnancies": 6, 
     "Glucose": 148, 
     "BloodPressure": 72, 
     "SkinThickness": 35, 
     "Insulin": 0, 
     "BMI": 33.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 50},

    {"Pregnancies": 1, 
     "Glucose": 85, 
     "BloodPressure": 66, 
     "SkinThickness": 29, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 31},
      ]}

body = str.encode(json.dumps(data))

url = 'http://a172e11b-9cad-4f30-83bf-5daf69628f52.southcentralus.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))
    print("Case 0: Not Diabetes, Case 1: Diabetes.")

In [ ]:
import requests
import json

# URL for the web service, should be similar to:
scoring_uri = 'http://0d469fee-33c0-4ce9-9dac-2767356110c9.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'zuv0yY9prOPHFssuYSFWDJPhnVMxgJqG'
#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 6, 
     "Glucose": 148, 
     "BloodPressure": 72, 
     "SkinThickness": 35, 
     "Insulin": 0, 
     "BMI": 33.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 50},

    {"Pregnancies": 1, 
     "Glucose": 85, 
     "BloodPressure": 66, 
     "SkinThickness": 29, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 31},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Not Diabetes, Case 1: Diabetes.")

In [ ]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
output

Print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()
compute_target.delete()